#  Vibe Matcher - AI-Powered Fashion Recommendation System

## Introduction

**Why AI at Nexora?**

AI represents the future of personalized fashion discovery. At Nexora, leveraging AI-powered recommendation systems allows us to transcend traditional keyword-based search, enabling customers to find products that match their aesthetic feelings and emotional connections to fashion. By using semantic embeddings and vector similarity, we can understand the *vibe* behind a customer's query—whether they're seeking "cozy minimalist comfort" or "bold statement pieces"—and deliver highly relevant product recommendations that resonate with their unique style identity. This prototype demonstrates how modern NLP techniques can bridge the gap between abstract fashion concepts and concrete product matches, creating a more intuitive and satisfying shopping experience.

---


##  Setup & Imports

In [ ]:
!pip install openai pandas scikit-learn matplotlib numpy python-dotenv seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import time
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_colwidth', None)
sns.set_style('whitegrid')

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Data Preparation


In [ ]:
products_data = [
    {
        "product_id": "P001",
        "name": "Boho Maxi Dress",
        "description": "Flowy maxi dress in earthy tones with intricate embroidery. Perfect for festival vibes, sunset gatherings, and free-spirited adventures. Features bohemian prints and a relaxed silhouette.",
        "price": 89.99,
        "category": "Dresses",
        "vibes": ["boho", "festival", "earthy", "relaxed", "artistic"]
    },
    {
        "product_id": "P002",
        "name": "Urban Leather Jacket",
        "description": "Sleek black leather jacket with edgy asymmetric zipper. Embodies energetic urban chic with a rebellious attitude. Perfect for night outs and street style statements.",
        "price": 249.99,
        "category": "Outerwear",
        "vibes": ["urban", "edgy", "energetic", "bold", "street-style"]
    },
    {
        "product_id": "P003",
        "name": "Cozy Oversized Sweater",
        "description": "Ultra-soft cashmere blend oversized sweater in warm cream. Minimalist design for cozy vibes, lazy Sundays, and comfort-first aesthetics. Pairs perfectly with your favorite loungewear.",
        "price": 119.99,
        "category": "Tops",
        "vibes": ["cozy", "minimalist", "comfort", "soft", "relaxed"]
    },
    {
        "product_id": "P004",
        "name": "Statement Sequin Blazer",
        "description": "Bold emerald green blazer covered in shimmering sequins. Make a dramatic entrance with this glamorous, attention-grabbing piece. Designed for confident, show-stopping moments.",
        "price": 199.99,
        "category": "Blazers",
        "vibes": ["bold", "glamorous", "statement", "confident", "dramatic"]
    },
    {
        "product_id": "P005",
        "name": "Minimalist Linen Pants",
        "description": "Clean-cut high-waisted linen pants in soft beige. Effortless elegance meets sustainable fashion. Perfect for sophisticated casual looks with a zen, understated aesthetic.",
        "price": 79.99,
        "category": "Bottoms",
        "vibes": ["minimalist", "elegant", "sustainable", "zen", "sophisticated"]
    },
    {
        "product_id": "P006",
        "name": "Vintage Denim Jacket",
        "description": "Classic light-wash denim jacket with distressed details and retro patches. Nostalgic, casual, and effortlessly cool. A timeless staple for laid-back, vintage-inspired outfits.",
        "price": 89.99,
        "category": "Outerwear",
        "vibes": ["vintage", "casual", "retro", "nostalgic", "timeless"]
    },
    {
        "product_id": "P007",
        "name": "Athletic Windbreaker",
        "description": "Lightweight neon-accented windbreaker with moisture-wicking technology. High-energy sporty vibes for active lifestyles. Perfect for runners, gym enthusiasts, and adventure seekers.",
        "price": 69.99,
        "category": "Activewear",
        "vibes": ["sporty", "energetic", "athletic", "active", "dynamic"]
    },
    {
        "product_id": "P008",
        "name": "Romantic Silk Blouse",
        "description": "Delicate blush pink silk blouse with pearl buttons and ruffled details. Soft, feminine, and romantic. Ideal for date nights, garden parties, and dreamy occasions.",
        "price": 139.99,
        "category": "Tops",
        "vibes": ["romantic", "feminine", "delicate", "soft", "dreamy"]
    },
    {
        "product_id": "P009",
        "name": "Edgy Cargo Pants",
        "description": "Black utility cargo pants with multiple pockets and chain details. Urban streetwear meets functional fashion. For those who embrace bold, utilitarian style.",
        "price": 99.99,
        "category": "Bottoms",
        "vibes": ["edgy", "urban", "utilitarian", "streetwear", "bold"]
    },
    {
        "product_id": "P010",
        "name": "Tropical Print Shirt",
        "description": "Vibrant Hawaiian-style shirt with palm leaves and sunset colors. Fun, vacation-ready vibes with a carefree, tropical spirit. Perfect for beach days and summer adventures.",
        "price": 59.99,
        "category": "Tops",
        "vibes": ["tropical", "fun", "vacation", "vibrant", "carefree"]
    }
]

df_products = pd.DataFrame(products_data)

print("🛍️ Fashion Product Catalog")
print("=" * 80)
print(f"Total Products: {len(df_products)}\n")
df_products[['product_id', 'name', 'price', 'category', 'vibes']]

🛍️ Fashion Product Catalog
Total Products: 10



,product_id,name,price,category,vibes
0,P001,Boho Maxi Dress,89.99,Dresses,"[boho, festival, earthy, relaxed, artistic]"
1,P002,Urban Leather Jacket,249.99,Outerwear,"[urban, edgy, energetic, bold, street-style]"
2,P003,Cozy Oversized Sweater,119.99,Tops,"[cozy, minimalist, comfort, soft, relaxed]"
3,P004,Statement Sequin Blazer,199.99,Blazers,"[bold, glamorous, statement, confident, dramatic]"
4,P005,Minimalist Linen Pants,79.99,Bottoms,"[minimalist, elegant, sustainable, zen, sophisticated]"
5,P006,Vintage Denim Jacket,89.99,Outerwear,"[vintage, casual, retro, nostalgic, timeless]"
6,P007,Athletic Windbreaker,69.99,Activewear,"[sporty, energetic, athletic, active, dynamic]"
7,P008,Romantic Silk Blouse,139.99,Tops,"[romantic, feminine, delicate, soft, dreamy]"
8,P009,Edgy Cargo Pants,99.99,Bottoms,"[edgy, urban, utilitarian, streetwear, bold]"
9,P010,Tropical Print Shirt,59.99,Tops,"[tropical, fun, vacation, vibrant, carefree]"


In [5]:
# Display detailed product information
print("\n📝 Product Details:\n")
for _, product in df_products.iterrows():
    print(f"{'='*80}")
    print(f"🏷️  {product['name']} ({product['product_id']})")
    print(f"💰 Price: ${product['price']}")
    print(f"📂 Category: {product['category']}")
    print(f"✨ Vibes: {', '.join(product['vibes'])}")
    print(f"📄 Description: {product['description']}")
    print()


📝 Product Details:

🏷️  Boho Maxi Dress (P001)
💰 Price: $89.99
📂 Category: Dresses
✨ Vibes: boho, festival, earthy, relaxed, artistic
📄 Description: Flowy maxi dress in earthy tones with intricate embroidery. Perfect for festival vibes, sunset gatherings, and free-spirited adventures. Features bohemian prints and a relaxed silhouette.

🏷️  Urban Leather Jacket (P002)
💰 Price: $249.99
📂 Category: Outerwear
✨ Vibes: urban, edgy, energetic, bold, street-style
📄 Description: Sleek black leather jacket with edgy asymmetric zipper. Embodies energetic urban chic with a rebellious attitude. Perfect for night outs and street style statements.

🏷️  Cozy Oversized Sweater (P003)
💰 Price: $119.99
📂 Category: Tops
✨ Vibes: cozy, minimalist, comfort, soft, relaxed
📄 Description: Ultra-soft cashmere blend oversized sweater in warm cream. Minimalist design for cozy vibes, lazy Sundays, and comfort-first aesthetics. Pairs perfectly with your favorite loungewear.

🏷️  Statement Sequin Blazer (P004)
💰 P

## 🤖 Embeddings Generation

Using OpenAI's `text-embedding-ada-002` model to generate vector embeddings for product descriptions.

In [ ]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
print("✅ OpenAI client initialized successfully!")

✅ OpenAI client initialized successfully!


In [ ]:
def get_embedding(text: str, model: str = "text-embedding-ada-002", use_mock: bool = False) -> List[float]:
    if use_mock:
        np.random.seed(hash(text) % (2**32))
        return np.random.randn(1536).tolist()
    
    try:
        text = text.replace("\n", " ").strip()
        response = client.embeddings.create(
            input=[text],
            model=model
        )
        return response.data[0].embedding
    
    except Exception as e:
        error_msg = str(e)
        if "429" in error_msg or "quota" in error_msg.lower():
            print(f"⚠️ API quota exceeded. Switching to mock embeddings...")
            return get_embedding(text, model, use_mock=True)
        else:
            print(f"❌ Error generating embedding: {e}")
            return None

print("✅ Embedding function defined!")

✅ Embedding function defined!


In [ ]:
print("🔄 Generating embeddings for product descriptions...\n")

embeddings_list = []
start_time = time.time()
failed_count = 0

for idx, product in df_products.iterrows():
    print(f"Processing {idx+1}/{len(df_products)}: {product['name']}...", end=" ")
    
    embedding_start = time.time()
    embedding = get_embedding(product['description'])
    embedding_time = time.time() - embedding_start
    
    if embedding is None:
        print(f"❌ Failed")
        failed_count += 1
        embedding = get_embedding(product['description'], use_mock=True)
    else:
        print(f"✓ ({embedding_time:.2f}s)")
    
    embeddings_list.append(embedding)
    time.sleep(0.2)

total_time = time.time() - start_time
df_products['embedding'] = embeddings_list

print(f"\n{'='*80}")
if failed_count > 0:
    print(f"⚠️ {failed_count} embeddings generated using mock data (API quota exceeded)")
    print(f"💡 Mock embeddings are deterministic and will work for demonstration purposes.")
else:
    print(f"✅ All embeddings generated successfully using OpenAI API!")

print(f"⏱️ Total time: {total_time:.2f} seconds")
print(f"📊 Embedding dimensions: {len(embeddings_list[0])}")
print(f"{'='*80}")

 Generating embeddings for product descriptions...

 NOTE: If you see API quota errors, the system will automatically switch to mock embeddings.

Processing 1/10: Boho Maxi Dress... ❌ Error generating embedding: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
 Failed


TypeError: get_embedding() got an unexpected keyword argument 'use_mock'

## 📊 Data Splitting - Training & Testing

Splitting the product catalog into training and testing sets to evaluate model performance.

In [ ]:
from sklearn.model_selection import train_test_split

train_size = 0.7
test_size = 0.3

df_train, df_test = train_test_split(
    df_products, 
    test_size=test_size, 
    random_state=42,
    shuffle=True
)

print(f"{'='*80}")
print(f" Data Split Summary")
print(f"{'='*80}")
print(f"Total Products: {len(df_products)}")
print(f"Training Set: {len(df_train)} products ({train_size*100:.0f}%)")
print(f"Testing Set: {len(df_test)} products ({test_size*100:.0f}%)")
print(f"{'='*80}\n")

print("🎯 Training Products:")
for idx, row in df_train.iterrows():
    print(f"  - {row['name']} (ID: {row['product_id']})")

print(f"\n🧪 Testing Products:")
for idx, row in df_test.iterrows():
    print(f"  - {row['name']} (ID: {row['product_id']})")

##  Model Training

In [ ]:
train_embeddings = np.array(df_train['embedding'].tolist())
train_product_ids = df_train['product_id'].tolist()
train_names = df_train['name'].tolist()

print(f"{'='*80}")
print(f"🎓 Training Model")
print(f"{'='*80}")
print(f"Training embeddings shape: {train_embeddings.shape}")
print(f"Embedding dimension: {train_embeddings.shape[1]}")
print(f"Number of training samples: {train_embeddings.shape[0]}")
print(f"{'='*80}\n")

print("✅ Model trained successfully!")
print("💡 The model uses cosine similarity to match query embeddings with product embeddings.")

##  Model Testing & Evaluation


In [ ]:
def evaluate_on_test_set(df_train, df_test, top_k=3):
    test_results = []
    
    print(f"{'='*80}")
    print(f"🧪 Testing Model on Unseen Data")
    print(f"{'='*80}\n")
    
    for idx, test_product in df_test.iterrows():
        test_embedding = np.array(test_product['embedding']).reshape(1, -1)
        train_embeddings_matrix = np.array(df_train['embedding'].tolist())
        
        similarities = cosine_similarity(test_embedding, train_embeddings_matrix)[0]
        
        df_train_copy = df_train.copy()
        df_train_copy['similarity_score'] = similarities
        df_train_copy = df_train_copy.sort_values('similarity_score', ascending=False)
        
        top_matches = df_train_copy.head(top_k)
        top_score = similarities.max()
        avg_score = similarities.mean()
        
        test_results.append({
            'test_product': test_product['name'],
            'test_vibes': test_product['vibes'],
            'top_match': top_matches.iloc[0]['name'],
            'top_match_vibes': top_matches.iloc[0]['vibes'],
            'top_score': top_score,
            'avg_score': avg_score
        })
        
        print(f"Test Product: {test_product['name']}")
        print(f"  Vibes: {', '.join(test_product['vibes'])}")
        print(f"  Top Match: {top_matches.iloc[0]['name']} (Score: {top_score:.4f})")
        print(f"  Match Vibes: {', '.join(top_matches.iloc[0]['vibes'])}")
        print()
    
    return pd.DataFrame(test_results)

df_test_results = evaluate_on_test_set(df_train, df_test, top_k=3)

print(f"{'='*80}")
print(f"✅ Testing Complete!")
print(f"{'='*80}")

In [ ]:
print(f"\n📊 Test Set Performance Metrics")
print(f"{'='*80}")
print(f"Average Top Match Score: {df_test_results['top_score'].mean():.4f}")
print(f"Min Top Match Score: {df_test_results['top_score'].min():.4f}")
print(f"Max Top Match Score: {df_test_results['top_score'].max():.4f}")
print(f"Standard Deviation: {df_test_results['top_score'].std():.4f}")
print(f"{'='*80}\n")

df_test_results[['test_product', 'top_match', 'top_score']]

## 📈 Training vs Testing Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Training vs Testing Performance', fontsize=16, fontweight='bold')

ax1 = axes[0]
datasets = ['Training Set', 'Testing Set']
counts = [len(df_train), len(df_test)]
colors = ['#3498db', '#e74c3c']
ax1.bar(datasets, counts, color=colors, alpha=0.7, edgecolor='black')
ax1.set_title('Dataset Size Comparison', fontweight='bold')
ax1.set_ylabel('Number of Products')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(counts):
    ax1.text(i, v + 0.1, str(v), ha='center', fontweight='bold')

ax2 = axes[1]
test_scores = df_test_results['top_score'].tolist()
ax2.hist(test_scores, bins=10, color='#2ecc71', edgecolor='black', alpha=0.7)
ax2.set_title('Test Set Similarity Score Distribution', fontweight='bold')
ax2.set_xlabel('Similarity Score')
ax2.set_ylabel('Frequency')
ax2.axvline(x=df_test_results['top_score'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f"Mean: {df_test_results['top_score'].mean():.3f}")
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Training vs Testing visualization complete!")

## 🔍 Vector Search & Similarity Matching

Implementing cosine similarity-based product matching.

In [ ]:
def find_similar_products(
    query: str, 
    df: pd.DataFrame, 
    top_k: int = 3,
    threshold: float = 0.0
) -> Tuple[pd.DataFrame, List[float], float]:
    start_time = time.time()
    query_embedding = get_embedding(query)
    
    if query_embedding is None:
        print("❌ Failed to generate query embedding")
        return None, [], 0.0
    
    query_embedding = np.array(query_embedding).reshape(1, -1)
    product_embeddings = np.array(df['embedding'].tolist())
    similarities = cosine_similarity(query_embedding, product_embeddings)[0]
    
    df_results = df.copy()
    df_results['similarity_score'] = similarities
    df_results = df_results.sort_values('similarity_score', ascending=False)
    df_results = df_results[df_results['similarity_score'] >= threshold]
    
    top_results = df_results.head(top_k)
    top_scores = top_results['similarity_score'].tolist()
    latency = time.time() - start_time
    
    return top_results, top_scores, latency

print("✅ Similarity search function defined!")

In [ ]:
def display_results(query: str, results: pd.DataFrame, scores: List[float], latency: float):
    print(f"\n{'='*80}")
    print(f"🔍 QUERY: \"{query}\"")
    print(f"⏱️  Processing Time: {latency:.3f} seconds")
    print(f"{'='*80}\n")
    
    if len(results) == 0:
        print("⚠️  No matches found above the threshold.")
        print("💡 Fallback: Try rephrasing your query or explore our featured collections!")
        return
    
    for idx, (_, product) in enumerate(results.iterrows(), 1):
        score = product['similarity_score']
        
        if score >= 0.8:
            quality = "🟢 Excellent Match"
        elif score >= 0.7:
            quality = "🟡 Good Match"
        elif score >= 0.6:
            quality = "🟠 Moderate Match"
        else:
            quality = "🔴 Weak Match"
        
        print(f"#{idx} - {quality} (Score: {score:.4f})")
        print(f"   🏷️  {product['name']} ({product['product_id']})")
        print(f"   💰 ${product['price']}")
        print(f"   ✨ Vibes: {', '.join(product['vibes'])}")
        print(f"   📝 {product['description'][:100]}...")
        print()

print("✅ Display function defined!")

## 🧪 Testing & Evaluation

Running multiple test queries and evaluating performance.

In [ ]:
test_queries = [
    "energetic urban chic",
    "cozy comfortable weekend vibes",
    "bold statement pieces for a night out",
    "romantic and feminine date night outfit",
    "casual vintage aesthetic"
]

print("🧪 Running Test Queries...\n")

In [ ]:
query_metrics = []

for query in test_queries:
    results, scores, latency = find_similar_products(
        query=query,
        df=df_products,
        top_k=3,
        threshold=0.0
    )
    
    display_results(query, results, scores, latency)
    
    query_metrics.append({
        'query': query,
        'top_score': scores[0] if scores else 0,
        'avg_score': np.mean(scores) if scores else 0,
        'min_score': scores[-1] if scores else 0,
        'latency': latency,
        'num_results': len(results)
    })
    
    time.sleep(0.5)

print("\n✅ All test queries completed!")

In [ ]:
df_metrics = pd.DataFrame(query_metrics)

print("\n📊 Query Performance Metrics")
print("=" * 80)
df_metrics

In [ ]:
# Calculate summary statistics
print("\n📈 Summary Statistics")
print("=" * 80)
print(f"Average Top Score: {df_metrics['top_score'].mean():.4f}")
print(f"Average Latency: {df_metrics['latency'].mean():.4f} seconds")
print(f"Max Latency: {df_metrics['latency'].max():.4f} seconds")
print(f"Min Latency: {df_metrics['latency'].min():.4f} seconds")
print(f"\nQueries with 'Good' matches (>0.7): {len(df_metrics[df_metrics['top_score'] > 0.7])} / {len(df_metrics)}")
print(f"Queries with 'Excellent' matches (>0.8): {len(df_metrics[df_metrics['top_score'] > 0.8])} / {len(df_metrics)}")

## 📊 Visualization & Analysis

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Vibe Matcher Performance Analysis', fontsize=16, fontweight='bold')

# 1. Similarity Scores Distribution
ax1 = axes[0, 0]
df_metrics[['top_score', 'avg_score', 'min_score']].plot(kind='bar', ax=ax1, color=['#2ecc71', '#3498db', '#e74c3c'])
ax1.set_title('Similarity Scores by Query', fontweight='bold')
ax1.set_xlabel('Query Index')
ax1.set_ylabel('Similarity Score')
ax1.axhline(y=0.7, color='orange', linestyle='--', label='Good Threshold (0.7)')
ax1.legend(['Top Score', 'Avg Score', 'Min Score', 'Threshold'])
ax1.grid(axis='y', alpha=0.3)

# 2. Latency Analysis
ax2 = axes[0, 1]
ax2.bar(range(len(df_metrics)), df_metrics['latency'], color='#9b59b6')
ax2.set_title('Query Processing Latency', fontweight='bold')
ax2.set_xlabel('Query Index')
ax2.set_ylabel('Latency (seconds)')
ax2.axhline(y=df_metrics['latency'].mean(), color='red', linestyle='--', label=f"Avg: {df_metrics['latency'].mean():.3f}s")
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Score Distribution Histogram
ax3 = axes[1, 0]
all_scores = df_metrics['top_score'].tolist() + df_metrics['avg_score'].tolist() + df_metrics['min_score'].tolist()
ax3.hist(all_scores, bins=15, color='#1abc9c', edgecolor='black', alpha=0.7)
ax3.set_title('Overall Score Distribution', fontweight='bold')
ax3.set_xlabel('Similarity Score')
ax3.set_ylabel('Frequency')
ax3.axvline(x=0.7, color='orange', linestyle='--', linewidth=2, label='Good Threshold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. Performance Quality Breakdown
ax4 = axes[1, 1]
quality_counts = {
    'Excellent (>0.8)': len(df_metrics[df_metrics['top_score'] > 0.8]),
    'Good (0.7-0.8)': len(df_metrics[(df_metrics['top_score'] >= 0.7) & (df_metrics['top_score'] <= 0.8)]),
    'Moderate (0.6-0.7)': len(df_metrics[(df_metrics['top_score'] >= 0.6) & (df_metrics['top_score'] < 0.7)]),
    'Weak (<0.6)': len(df_metrics[df_metrics['top_score'] < 0.6])
}
colors_pie = ['#2ecc71', '#f39c12', '#e67e22', '#e74c3c']
ax4.pie(quality_counts.values(), labels=quality_counts.keys(), autopct='%1.1f%%', 
        colors=colors_pie, startangle=90)
ax4.set_title('Match Quality Distribution', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Visualizations generated successfully!")

##  Edge Case Handling

In [ ]:
print("🧪 Testing Edge Cases...\n")

edge_cases = [
    "quantum physics laboratory equipment",
    "xyz123 nonsense random words",
    "",
]

for query in edge_cases:
    if not query.strip():
        print(f"{'='*80}")
        print("🔍 QUERY: [Empty String]")
        print("⚠️  Empty query detected!")
        print("💡 Fallback: Please provide a description of the style you're looking for.")
        print(f"{'='*80}\n")
        continue
    
    results, scores, latency = find_similar_products(
        query=query,
        df=df_products,
        top_k=3,
        threshold=0.65
    )
    
    display_results(query, results, scores, latency)
    
    if len(results) == 0 or (scores and scores[0] < 0.65):
        print("💡 Fallback Recommendation:")
        print("   - Check our trending collections")
        print("   - Browse by category")
        print("   - Try a different search term\n")

##  Interactive Demo



In [ ]:
def search_products(query: str, top_k: int = 3, min_score: float = 0.6):
    if not query.strip():
        print("  Please provide a query!")
        return
    
    results, scores, latency = find_similar_products(
        query=query,
        df=df_products,
        top_k=top_k,
        threshold=min_score
    )
    
    display_results(query, results, scores, latency)

print("✅ Interactive search function ready!")
print("\n💡 Try it out:")
print('   search_products("your vibe here")')